In [ ]:
import glob
from fps.fps import farthest_point_sample, index_points
import os
import torch
import numpy as np
import all_tools

In [ ]:
def fps_downsample(data_path=None):
    point_num=10000
    print(data_path)
    data = all_tools.read_ply2np(fr"{data_path}")
    xyz = data[:, :3]
    xyz = xyz[np.newaxis, :, :]
    xyz = torch.from_numpy(xyz).to("cuda")
    centroids = farthest_point_sample(xyz, point_num)
    index = centroids[0].cpu().numpy()
    fps_pcd = data[index]
    save_path = data_path.replace("Crops3D","Crops3D_10k")
    save_path = save_path.replace(".ply","_10k.ply")
    all_tools.save_ply_from_np(fps_pcd, save_path)
    return

In [ ]:
# Create a mirror directory Crops3D_10k
src_dir = r"Path\Crops3D"  # Please replace with your source directory
dst_dir = r"Path\Crops3D_10k"  # Please replace with your destination directory

# Traverse all subdirectories in the source directory
for subdir, dirs, _ in os.walk(src_dir):
    # Get the corresponding subdirectory path in the destination directory
    dst_subdir = subdir.replace(src_dir, dst_dir)
    # Create the subdirectory in the destination directory if it doesn't exist
    if not os.path.exists(dst_subdir):
        os.makedirs(dst_subdir)

In [ ]:
from multiprocessing_on_dill.dummy import Pool

# Batch processing program
ply_files = glob.glob(os.path.join(r"Path\Crops3D", '**/*.ply'), recursive=True)

pool = Pool(5)
pool.map(fps_downsample, ply_files)
pool.close()
pool.join()